In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pandas as pd

import sklearn
from sklearn.model_selection import train_test_split
from collections import OrderedDict

from sklearn.ensemble import RandomForestRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, Matern

from sklearn.metrics import r2_score

from sklearn.linear_model import LinearRegression

from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
import random

from sklearn.preprocessing import StandardScaler

In [2]:
!python --version

Python 3.7.4


In [3]:
print(np.__version__,"numpy")
print(pd.__version__,"pandas")
print(sklearn.__version__,"sklearn")

1.16.5 numpy
1.1.5 pandas
0.21.3 sklearn


In [4]:
SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(SEED)

def make_t_scale_df(df):
    
    cols = [x for x in df.columns if x not in ["target","mutant"]]
    for t,i in enumerate(cols):
        df = pd.merge(df,config.t_scale, how = "left",left_on= i , right_on= "code")
        df.drop("code",axis = 1, inplace = True)
        df.drop("As",axis = 1, inplace = True)
        keep_cols =  ["t1","t2","t3","t4","t5"]
        df.rename(columns = {x: str(x) + "_" + str(t + 1) for x in keep_cols},inplace = True)
    df.drop(cols,axis = 1, inplace = True)
    
    return df

def top_k(df,y_true = "target",y_hat = "pred",k = 12):
    y_true = df.sort_values("target",ascending = False)[:k]["mutant"].values
    y_hat = df.sort_values("pred",ascending = False)[:k]["mutant"].values
    in_common = list(set(y_true).intersection(y_hat))
    return len(in_common) / k

In [5]:
def make_dataframes(df,newinfo = None):
    """
    input a dataframe of mutants, returns a preprocessed dataframes for measured and remaining mutants
    
    :df: reqires a column with mutants
    :add_newinfo: bool, whether to add additional information based on AS properties
    
    """
    
    df_ml = df[["mutant","target"]].reset_index(drop = True)

    df_ml["V81"] = df_ml.apply(lambda x: str(x["mutant"])[0],axis = 1)
    df_ml["A88"] = df_ml.apply(lambda x: str(x["mutant"])[1],axis = 1)
    df_ml["I161"] = df_ml.apply(lambda x: str(x["mutant"])[2],axis = 1)
    
    all_aminoacids = np.array([np.array([i,j,k]) for i in  df_ml["V81"].unique() for j in  df_ml["V81"].unique() for k in  df_ml["V81"].unique()])
    all_aminoacids_df = pd.DataFrame({"V81":all_aminoacids[:,0],"A88":all_aminoacids[:,1],"I161":all_aminoacids[:,2]})
    
    if len(df_ml["V81"].unique()) < 20:
        print("error, not all amino acids covered")
    
    all_aminoacids_df["mutant"] = all_aminoacids_df.apply(lambda x: x["V81"] + x["A88"] + x["I161"],axis = 1)
    all_aminoacids_df = all_aminoacids_df[~all_aminoacids_df["mutant"].isin(df_ml["mutant"])].reset_index(drop = True)

    df_ml2 = make_t_scale_df(df_ml)
    all_aminoacids_df2 = make_t_scale_df(all_aminoacids_df)
    
    all_aminoacids_df = all_aminoacids_df.drop("mutant",axis = 1)
    df_ml.drop("mutant",axis = 1,inplace = True)
    
    df_ml = pd.concat([df_ml.drop("target",axis = 1),df_ml2],axis = 1)
    all_aminoacids_df = pd.concat([all_aminoacids_df,all_aminoacids_df2],axis = 1)

    if newinfo is not None:
        for t,AS in enumerate(["V81","A88","I161"]):
            df_ml = pd.merge(df_ml,newinfo, how = "left", left_on = AS,right_on = "Code")

            all_aminoacids_df = pd.merge(all_aminoacids_df,newinfo, how = "left", left_on = AS,right_on = "Code")

            newcols = {x : x + "_" +str(AS) for x in newinfo.columns[1:]}

            df_ml.rename(columns = newcols, inplace = True)
            all_aminoacids_df.rename(columns = newcols, inplace = True)

            all_aminoacids_df.drop("Code",axis = 1, inplace = True)
            df_ml.drop("Code",axis = 1, inplace = True)
            
    return df_ml, all_aminoacids_df


def train(df_ml,all_amino_acids,n_splits):
    """
    simple training loop
    :df_ml: training samples
    :all_amino_acids: dataframe with remaining combinations to be predicted
    :n_splits: amount of cv splits
    
    returns oof predictions and predictions on remaining amino acids
    
    """
    n_splits = n_splits
    cv = KFold(n_splits = n_splits)

    oof_df = np.zeros(len(df_ml))
    y_preds_all = np.zeros(len(all_aminoacids_df))

    for t, (train_idx, val_idx) in enumerate(cv.split(df_ml)):
        print(f"fold {t}")

        X_train, X_test, y_train, y_test = df_ml.iloc[train_idx],df_ml.iloc[val_idx],df_ml["target"].iloc[train_idx],df_ml["target"].iloc[val_idx]
        
        X_train["target"] = y_train
        
        remaining_as = all_aminoacids_df
        
        features = [x for x in X_train.columns if x not in ["V81","A88","I161","target","mutant"]]

        lreg = GaussianProcessRegressor(kernel = Matern())
        lreg.fit(X_train[features],y_train)
        print(X_train[features].shape)
        yhat = np.expm1(lreg.predict(X_test[features])).clip(0,20)
        y_preds_all += np.expm1(lreg.predict(remaining_as[features])).clip(0,20) / n_splits

        oof_df[val_idx] = yhat

        print(r2_score(np.expm1(y_test),yhat))

    print("final:")
    print(r2_score(np.expm1(df_ml.target),oof_df))

    df_ml["pred"] = oof_df
    df_ml["target"] = np.expm1(df_ml["target"])

    print(f"top_k: {top_k(df_ml,k = 20)}")
    
    all_aminoacids_df["pred"] = y_preds_all
    
    return df_ml, all_aminoacids_df

In [6]:
class config:
    t_scale = pd.read_excel("t_scale.xlsx")
    splits = 10

In [7]:
df = pd.read_excel("training_data_activity.xlsx")
df.rename(columns = {"activity":"target"},inplace = True)

In [8]:
newinfo = pd.read_excel("table_Aa_phys_char.xlsx")

newinfo.drop(["Formula","Amino acid"],axis = 1, inplace = True)

cat_cols = ["Polarity","Acidity"]

for t,AS in enumerate(cat_cols):
    newinfo = pd.concat([newinfo, pd.get_dummies(newinfo[AS])] ,axis = 1)

newinfo.drop(cat_cols,axis = 1, inplace = True)

for t in ["Molar mass","Van der Waals volume","VR A3","Hydropathy index"]:
    sclr = MinMaxScaler()
    newinfo[t] = sclr.fit_transform(np.array(newinfo[t].astype(np.int64)).reshape((-1,1)))

In [9]:
df_ml,all_aminoacids_df = make_dataframes(df,newinfo)

df_ml["target"] = np.log1p(df_ml["target"])

df_ml, all_aminoacids_df = train(df_ml,all_aminoacids_df,config.splits)

fold 0
(453, 51)
0.7010691017668559
fold 1
(453, 51)
-0.449463286855351
fold 2
(453, 51)
0.4698810338149775
fold 3
(453, 51)
0.8456292533858286
fold 4
(454, 51)
0.6948312573403558
fold 5
(454, 51)
0.8856408258570847
fold 6
(454, 51)
0.7172809209182913
fold 7
(454, 51)
0.5809533221812361
fold 8
(454, 51)
0.6073783243278099
fold 9
(454, 51)
0.6714144871757712
final:
0.7451655352769679
top_k: 0.75


In [10]:
all_aminoacids_df.sort_values("pred",ascending = False)[:10]

,V81,A88,I161,mutant,t1_1,t2_1,t3_1,t4_1,t5_1,t1_2,...,Hydropathy index_I161,pI_I161,Nonpolar_I161,Polar_I161,Acidic_I161,Basic_I161,Basic (strong)_I161,Basic (weak)_I161,Neutral_I161,pred
2358,S,I,P,SIP,-7.44,-0.65,0.68,-0.17,1.58,-4.25,...,0.375,6.30,1,0,0,0,0,0,1,10.169406
4247,V,I,A,VIA,-5.87,-0.94,0.28,1.10,0.48,-4.25,...,0.625,6.01,1,0,0,0,0,0,1,9.267188
6091,A,I,P,AIP,-9.11,-1.63,0.63,1.04,2.26,-4.25,...,0.375,6.30,1,0,0,0,0,0,1,9.218166
6262,A,L,P,ALP,-9.11,-1.63,0.63,1.04,2.26,-4.38,...,0.375,6.30,1,0,0,0,0,0,1,8.182579
4612,C,I,A,CIA,-7.35,-0.86,-0.33,0.80,0.98,-4.25,...,0.625,6.01,1,0,0,0,0,0,1,8.003926
4604,C,I,P,CIP,-7.35,-0.86,-0.33,0.80,0.98,-4.25,...,0.375,6.30,1,0,0,0,0,0,1,7.842601
2470,S,V,P,SVP,-7.44,-0.65,0.68,-0.17,1.58,-5.87,...,0.375,6.30,1,0,0,0,0,0,1,7.597747
2364,S,I,A,SIA,-7.44,-0.65,0.68,-0.17,1.58,-4.25,...,0.625,6.01,1,0,0,0,0,0,1,7.383347
4767,C,L,P,CLP,-7.35,-0.86,-0.33,0.80,0.98,-4.38,...,0.375,6.30,1,0,0,0,0,0,1,7.231569
4416,V,L,A,VLA,-5.87,-0.94,0.28,1.10,0.48,-4.38,...,0.625,6.01,1,0,0,0,0,0,1,7.184152


In [11]:
df_ml = df_ml[["V81","A88","I161","mutant","pred","target"]]
all_aminoacids_df = all_aminoacids_df[["V81","A88","I161","mutant","pred"]]

In [12]:
results_activity = pd.concat([df_ml,all_aminoacids_df]).reset_index(drop = True)

In [13]:
results_activity = results_activity.rename(columns = {"pred":"predicted_activity","target":"measured_activity"})

In [14]:
results_activity

,V81,A88,I161,mutant,predicted_activity,measured_activity
0,G,Y,T,GYT,0.000000,0.0
1,Q,H,L,QHL,0.000000,0.0
2,K,W,H,KWH,0.002476,0.0
3,R,Y,T,RYT,0.000394,0.0
4,K,H,R,KHR,0.058775,0.0
...,...,...,...,...,...,...
7995,N,N,M,NNM,0.104063,NaN
7996,N,N,A,NNA,0.414223,NaN
7997,N,N,D,NND,0.003642,NaN
7998,N,N,Y,NNY,0.021950,NaN


In [15]:
### SELECTIVITY ###

In [16]:
newdf = pd.read_excel("training_data_selectivity.xlsx")
newdf.rename(columns = {"selectivity":"target"},inplace = True)

In [17]:
df_ml_sel,all_aminoacids_df = make_dataframes(newdf,newinfo)

n_splits = 10
cv = KFold(n_splits = n_splits,random_state = 42)
oof_df = np.zeros(len(df_ml_sel))
y_preds_all = np.zeros(len(all_aminoacids_df))

for t, (train_idx, val_idx) in enumerate(cv.split(df_ml_sel)):
    
    X_train, X_test, y_train, y_test = df_ml_sel.iloc[train_idx],df_ml_sel.iloc[val_idx],df_ml_sel["target"].iloc[train_idx],df_ml_sel["target"].iloc[val_idx]
    
    X_train["target"] = y_train
    
    features = [x for x in X_train.columns if x not in ["V81","A88","I161","target","mutant"]]
        
    lreg = RandomForestRegressor(random_state = 29)
    lreg.fit(X_train[features],y_train)

    yhat = lreg.predict(X_test[features]).clip(-1,1)
    y_preds_all += lreg.predict(all_aminoacids_df[features]).clip(-1,1) / n_splits
    
    oof_df[val_idx] = yhat
    
    
print("final:")
print(r2_score(df_ml_sel.target,oof_df))

df_ml_sel["pred"] = oof_df
df_ml_sel["mutant"] = df_ml_sel["mutant"]

df_ml_sel["target"] = df_ml_sel["target"]
all_aminoacids_df["pred"] = y_preds_all

print(f"top_k: {top_k(df_ml_sel,k = 20)}")

final:
0.3092606951775465
top_k: 0.6


In [18]:
df_ml_sel = df_ml_sel[["V81","A88","I161","mutant","pred","target"]]
all_aminoacids_df = all_aminoacids_df[["V81","A88","I161","mutant","pred"]]

results_selectivity= pd.concat([df_ml_sel,all_aminoacids_df]).reset_index(drop = True)

In [19]:
results_selectivity = results_selectivity.rename(columns = {"pred":"predicted_selectivity","target":"measured_selectivity"})

In [20]:
results = pd.merge(results_activity,results_selectivity[results_selectivity.columns[-3:]], how = "left", on = "mutant")

In [21]:
results.to_csv("results.csv",index = False)

In [22]:
results

,V81,A88,I161,mutant,predicted_activity,measured_activity,predicted_selectivity,measured_selectivity
0,G,Y,T,GYT,0.000000,0.0,-0.294721,NaN
1,Q,H,L,QHL,0.000000,0.0,-0.111410,NaN
2,K,W,H,KWH,0.002476,0.0,-0.720761,NaN
3,R,Y,T,RYT,0.000394,0.0,-0.401586,NaN
4,K,H,R,KHR,0.058775,0.0,-0.028869,NaN
...,...,...,...,...,...,...,...,...
7995,N,N,M,NNM,0.104063,NaN,-0.454452,NaN
7996,N,N,A,NNA,0.414223,NaN,-0.420328,NaN
7997,N,N,D,NND,0.003642,NaN,-0.262550,NaN
7998,N,N,Y,NNY,0.021950,NaN,-0.310331,NaN
